In [2]:
%matplotlib inline
import pandas as pd
import numpy as np
from sklearn import decomposition
import matplotlib.pyplot as plt
import DataHelper

e = DataHelper.eclass_data()

In [8]:
e.get_buffy_pre()

,anon_student_id,survey_id,duration,q01a,q01b,q27a,q27b,q03a,q03b,q22a,...,q04b,q14a,q14b,q29a,q29b,q07a,q07b,q31a,q31b,ResponseId
0,24620,00KRiRH91SoGB3T,802.0,5.0,5.0,3.0,5.0,5.0,5.0,5.0,...,5.0,3.0,5.0,3.0,1.0,1.0,5.0,5.0,5.0,R_RRJcHqWgYyk9tC1
1,24621,00KRiRH91SoGB3T,529.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,...,5.0,5.0,5.0,5.0,5.0,1.0,5.0,5.0,5.0,R_RRJcHqWgYyk9tC1
2,24622,00KRiRH91SoGB3T,507.0,5.0,5.0,5.0,5.0,3.0,5.0,3.0,...,5.0,5.0,5.0,5.0,5.0,1.0,3.0,5.0,5.0,R_RRJcHqWgYyk9tC1
3,24623,00KRiRH91SoGB3T,555.0,5.0,5.0,5.0,5.0,1.0,5.0,5.0,...,5.0,5.0,5.0,5.0,5.0,1.0,5.0,5.0,5.0,R_RRJcHqWgYyk9tC1
4,24624,00KRiRH91SoGB3T,591.0,5.0,5.0,5.0,5.0,3.0,1.0,5.0,...,5.0,5.0,5.0,NaN,NaN,1.0,5.0,5.0,5.0,R_RRJcHqWgYyk9tC1
5,24625,00KRiRH91SoGB3T,686.0,5.0,5.0,5.0,5.0,5.0,5.0,1.0,...,5.0,5.0,5.0,5.0,5.0,1.0,1.0,5.0,5.0,R_RRJcHqWgYyk9tC1
6,24626,00KRiRH91SoGB3T,469.0,5.0,5.0,5.0,5.0,3.0,5.0,1.0,...,5.0,5.0,5.0,5.0,5.0,1.0,5.0,5.0,5.0,R_RRJcHqWgYyk9tC1
7,24627,00KRiRH91SoGB3T,812.0,5.0,5.0,3.0,5.0,1.0,5.0,1.0,...,5.0,1.0,5.0,5.0,5.0,1.0,5.0,5.0,5.0,R_RRJcHqWgYyk9tC1
8,24628,00KRiRH91SoGB3T,580.0,5.0,5.0,5.0,5.0,3.0,1.0,5.0,...,5.0,3.0,5.0,5.0,5.0,3.0,3.0,5.0,5.0,R_RRJcHqWgYyk9tC1
9,24629,00KRiRH91SoGB3T,2533.0,5.0,5.0,5.0,5.0,5.0,5.0,3.0,...,5.0,5.0,5.0,5.0,5.0,1.0,5.0,5.0,5.0,R_RRJcHqWgYyk9tC1


# Figure 1 : Cumulative number of student responses to ECLASS

# Table 1: Summary statistics describing differences in responses between intro and bfy

# Table 2: Demographic responses for ECLASS data set`

# Figure 2: plot comparing response rates per survey

# Table 3: Institutional descriptors table

# Table 4: Table on course information survey

# Figure 3: Spearman corrleations for ECLASS survey

# Figure 4: Skree plot for PCA of ECLASS survey

# Figure 5: Eigenvalues of first three principal components